In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config('spark.jars.packages', 'com.crealytics:spark-excel_2.12:0.13.5') \
    .getOrCreate()

In [0]:
df = spark.read.format("com.crealytics.spark.excel") \
    .option("inferSchema", "true") \
    .option("header", "false") \
    .option("dataAddress", "'Dirty 5'!A1") \
    .load("/FileStore/tables/5__Jumbled_up_Customers_Details-1.xlsx")

display(df)

_c0
Name Hussein Hakeem Address Number 22 Fioye Crescent Surulere Lagos Age 17 Gender Male
Name Arojoye Samuel Address 11 Omolade Close Omole Estate Lagos Age 16 Gender Male
"Name Alex Ezurum Address 1 Adamu Lane, Abuja Age 14 Gender Male"
"Name Susan Nwaimo Address Number 58 Yaba Street, Kaduna State Age 16 Gender Female"
"Name Ajao Opeyemi Address No12 Olubunmi Street, Abeokuta Age 18 Gender Female"
"Name Banjoko Adebusola Address 34 Ngige Street, Ugheli, Delta Age 14 Gender Female"
"Name Muhammed Olabisi Address 13, ICAN road, Enugu Age 12 Gender Female"
"Name Oluwagbemi Mojisola Address ACCA Lane, Onitsha Age 13 Gender Female"


In [0]:
from pyspark.sql.functions import split, expr, array_position, col, size

In [0]:
splitted_df = df.withColumn("splitted_col", split(df["_c0"], " "))

In [0]:
splitted_df = splitted_df.drop('_c0')

In [0]:
splitted_df.show()

+--------------------+
|        splitted_col|
+--------------------+
|[Name, Hussein, H...|
|[Name, Arojoye, S...|
|[Name, Alex, Ezur...|
|[Name, Susan, Nwa...|
|[Name, Ajao, Opey...|
|[Name, Banjoko, A...|
|[Name, Muhammed, ...|
|[Name, Oluwagbemi...|
+--------------------+



In [0]:
import pyspark.sql.functions as F

In [0]:
# df.select(
#     array_position(df['splitted_col'], "Age")
# ).show()

# splitted_df.select(idx_address).show()

idx_name = array_position(splitted_df["splitted_col"], 'Name')
idx_address = array_position(splitted_df["splitted_col"], 'Address')
idx_age = array_position(splitted_df["splitted_col"], 'Age')
idx_gender = array_position(splitted_df["splitted_col"], 'Gender')

# slice(column, start_index, length)
splitted_df.select(
    expr("concat_ws(' ', slice(splitted_col, array_position(splitted_col, 'Name') + 1, array_position(splitted_col, 'Address') - array_position(splitted_col, 'Name') - 1))").alias("name"), 
    
    expr("concat_ws(' ', slice(splitted_col, array_position(splitted_col, 'Address') + 1, array_position(splitted_col, 'Age') - array_position(splitted_col, 'Address') - 1))").alias("address"),
    
    expr("concat_ws(' ', slice(splitted_col, array_position(splitted_col, 'Age') + 1, array_position(splitted_col, 'Gender') - array_position(splitted_col, 'Age') - 1))").alias("age"),
    
    col("splitted_col")[size(col("splitted_col")) -1 ].alias("gender")
).show(truncate=False)

# test = splitted_df.select(
#     expr(f"slice(splitted_col, 1, 4)").alias("name123")
# )
# test.show()
# splitted_df.select(
#     # expr("slice(splitted_df, 1, 3)").alias("test")
#     expr(f"concat_ws(' ', slice(splitted_col, {idx_name} + 1, {idx_address} - {idx_name} - 1))").alias("name1")
# ).show()

+-------------------+---------------------------------------+---+------+
|name               |address                                |age|gender|
+-------------------+---------------------------------------+---+------+
|Hussein Hakeem     |Number 22 Fioye Crescent Surulere Lagos|17 |Male  |
|Arojoye Samuel     |11 Omolade Close Omole Estate Lagos    |16 |Male  |
|Alex Ezurum        |1 Adamu Lane, Abuja                    |14 |Male  |
|Susan Nwaimo       |Number 58 Yaba Street, Kaduna State    |16 |Female|
|Ajao Opeyemi       |No12 Olubunmi Street, Abeokuta         |18 |Female|
|Banjoko Adebusola  |34 Ngige Street, Ugheli, Delta         |14 |Female|
|Muhammed Olabisi   |13, ICAN road, Enugu                   |12 |Female|
|Oluwagbemi Mojisola|ACCA Lane, Onitsha                     |13 |Female|
+-------------------+---------------------------------------+---+------+

